In [2]:
from googleapiclient.discovery import build
import pandas as pd
import googleapiclient.discovery
from IPython.display import JSON
import itertools

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
%matplotlib inline
import seaborn as sb
import imageio
#import isodate

#NLP
from wordcloud import WordCloud, STOPWORDS
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [36]:
api_key = ['']


In [ ]:
a

In [19]:
channel_names = [
    "Alex The Analyst",
    "Corey Schafer",
    "Ken Jee",
    "Mo Chen",
    "Luke Barousse",
    "Data Professor",
    "Tech With Tim",
    "Data Science Jay",
    "Nicholas Renotte",
    "StatQuest with Josh Starmer"
]

# Build YouTube API service
youtube = build('youtube', 'v3', developerKey=api_key)

# Dictionary to store channel names and their IDs
channel_ids = {}

for channel_name in channel_names:
    # Search for the channel
    request = youtube.search().list(
        part='snippet',
        q=channel_name,
        type='channel',
        maxResults=1
    )
    response = request.execute()

    # Extract and store the channel ID
    if response['items']:
        channel_id = response['items'][0]['id']['channelId']
        channel_ids[channel_name] = channel_id
        print(f"Channel: {channel_name} | ID: {channel_id}")
    else:
        print(f"Channel: {channel_name} not found.")

# Print all channel IDs
print(channel_ids)


Channel: Alex The Analyst | ID: UC7cs8q-gJRlGwj4A8OmCmXg
Channel: Corey Schafer | ID: UCCezIgC97PvUuR4_gbFUs5g
Channel: Ken Jee | ID: UCiT9RITQ9PW6BhXK0y2jaeg
Channel: Mo Chen | ID: UCDybamfye5An6p-j1t2YMsg
Channel: Luke Barousse | ID: UCLLw7jmFsvfIVaUFsLs8mlQ
Channel: Data Professor | ID: UCV8e2g4IWQqK71bbzGDEI4Q
Channel: Tech With Tim | ID: UC4JX40jDee_tINbkjycV4Sg
Channel: Data Science Jay | ID: UCcQx1UnmorvmSEZef4X7-6g
Channel: Nicholas Renotte | ID: UCHXa4OpASJEwrHrLeIzw7Yg
Channel: StatQuest with Josh Starmer | ID: UCtYLUTtgS3k1Fg4y5tAhLbw
{'Alex The Analyst': 'UC7cs8q-gJRlGwj4A8OmCmXg', 'Corey Schafer': 'UCCezIgC97PvUuR4_gbFUs5g', 'Ken Jee': 'UCiT9RITQ9PW6BhXK0y2jaeg', 'Mo Chen': 'UCDybamfye5An6p-j1t2YMsg', 'Luke Barousse': 'UCLLw7jmFsvfIVaUFsLs8mlQ', 'Data Professor': 'UCV8e2g4IWQqK71bbzGDEI4Q', 'Tech With Tim': 'UC4JX40jDee_tINbkjycV4Sg', 'Data Science Jay': 'UCcQx1UnmorvmSEZef4X7-6g', 'Nicholas Renotte': 'UCHXa4OpASJEwrHrLeIzw7Yg', 'StatQuest with Josh Starmer': 'UCtYLUTtgS3k

In [29]:
youtube_channel_ids = [
    "UC7cs8q-gJRlGwj4A8OmCmXg",
    "UCCezIgC97PvUuR4_gbFUs5g",
    "UCiT9RITQ9PW6BhXK0y2jaeg",
    "UCDybamfye5An6p-j1t2YMsg",
    "UCLLw7jmFsvfIVaUFsLs8mlQ",
    "UCV8e2g4IWQqK71bbzGDEI4Q",
    "UC4JX40jDee_tINbkjycV4Sg",
    "UCcQx1UnmorvmSEZef4X7-6g",
    "UCHXa4OpASJEwrHrLeIzw7Yg",
    "UCtYLUTtgS3k1Fg4y5tAhLbw"
]

print(youtube_channel_ids)

['UC7cs8q-gJRlGwj4A8OmCmXg', 'UCCezIgC97PvUuR4_gbFUs5g', 'UCiT9RITQ9PW6BhXK0y2jaeg', 'UCDybamfye5An6p-j1t2YMsg', 'UCLLw7jmFsvfIVaUFsLs8mlQ', 'UCV8e2g4IWQqK71bbzGDEI4Q', 'UC4JX40jDee_tINbkjycV4Sg', 'UCcQx1UnmorvmSEZef4X7-6g', 'UCHXa4OpASJEwrHrLeIzw7Yg', 'UCtYLUTtgS3k1Fg4y5tAhLbw']


In [30]:
api_service_name = "youtube"
api_version = "v3"
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=api_key)

In [31]:
def get_channel_stats(youtube, channel_ids):
    """
    Function: Gather interested channel stats from youtube creator's channel page
    
    INPUT:
    youtube - build object from googleapiclient.discovery
    channel_ids - (list) list of channel ids to be analyzed
    
    OUTPUT:
    all_data - (pandas dataframe) dataframe that consists of the following columns: channelName, publishDate, subscribers, views, totalVideos, playlistId
    """
    all_data = []

    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(channel_ids)
    )
    response = request.execute()
    
    #loop through items
    for item in response['items']:
        data = {'channelName': item['snippet']['title'],
                'publishDate': item['snippet']['publishedAt'],
                'subscribers': item['statistics']['subscriberCount'],
                'views': item['statistics']['viewCount'],
                'totalVideos': item['statistics']['videoCount'],
                'playlistId': item['contentDetails']['relatedPlaylists']['uploads']
        }
        all_data.append(data)
    all_data = pd.DataFrame(all_data)

    return(all_data)

In [32]:
def get_videos_ids(youtube, playlist_id):
    """
    Function: Gather videoIds from channel. 
    
    INPUT:
    youtube - Get credentials and create an API client/Initialise a Youtube API service object.
    playlist_ids - (list) list of playlist ids to be analyzed.
    
    OUTPUT:
    video_ids - (list) list of dictionary that contains all videoId for channel.
    """
    video_ids = []
    
    request = youtube.playlistItems().list(
        part="snippet, contentDetails",
        playlistId= playlist_id,
        maxResults = 50
    )
    
    response = request.execute()
    
    for item in response['items']:
        data = {
                'videoId': item['contentDetails']['videoId']
               }
        video_ids.append(data)
    
    next_page_token = response.get('nextPageToken')  
    while next_page_token is not None:
        request = youtube.playlistItems().list(
            part="snippet, contentDetails",
            playlistId= playlist_id,
            maxResults = 50,
            pageToken = next_page_token
            )
        response = request.execute()

        for item in response['items']:
            data = {
                    'videoId': item['contentDetails']['videoId']
               }
            video_ids.append(data)
        next_page_token = response.get('nextPageToken')
        
    return video_ids

In [33]:
def get_video_details(youtube, video_ids):
    """
    Function: Gather interested information from videos and store in dataframe. 
    
    INPUT:
    youtube - Get credentials and create an API client/Initialise a Youtube API service object.
    video_ids - (list) list of video ids.
    
    OUTPUT:
    video_df - (pandas dataframe) dataframe of video statistics. Includes columns:
                 'channelTitle', 'title', 'description', 'tags', 'publishedAt',
                 'viewCount', 'likeCount', 'favouriteCount', 'commentCount',
                 'duration', 'definition', 'caption'%
    """
    all_video_info = []
    for i in range(0,len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id= ','.join(video_ids[i:i+50])
        )
        response = request.execute()

        for video in response['items']:
            # create dictionary of stats I want to keep
            stats_keep = {'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                          'statistics': ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
                          'contentDetails': ['duration', 'definition', 'caption']
                        }
            
            # empty dictionary to keep track of keys and values
            video_info = {}
            video_info['video_id'] = video['id']
            
            # extract values and append them into empty dictionary
            for k in stats_keep.keys():
                for v in stats_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None

            all_video_info.append(video_info)
        video_df = pd.DataFrame(all_video_info)
    return video_df

In [34]:
def get_videos_comments(youtube, video_ids):
    """
    Function: Gather comments from videos and store in dataframe. 
    
    INPUT:
    youtube - Get credentials and create an API client/Initialise a Youtube API service object.
    video_ids - (list) list of video ids.
    
    OUTPUT:
    all_comments_df - (pandas dataframe) dataframe of comments. Each video has a max of 
                        10 comments which are compiled in a list.
    """
    all_comments = []
    
    for video_id in video_ids:
        try:
            request = youtube.commentThreads().list(
                part="snippet,replies",
                videoId=video_id
            )
            response = request.execute()
            # help https://developers.google.com/youtube/v3/docs/commentThreads?hl=en_US#snippet.topLevelComment
            video_comments = [comment['snippet']['topLevelComment']['snippet']['textOriginal'] 
                              for comment in response['items'][0:10]]
            video_comments_info = {'video_id': video_id, 'comments': video_comments}
            all_comments.append(video_comments_info)
        except:
            # dealing with errors
            pass 
            
    all_comments_df = pd.DataFrame(all_comments)
    return all_comments_df
        

In [35]:
channel_stats = get_channel_stats(youtube, youtube_channel_ids)
channel_stats

,channelName,publishDate,subscribers,views,totalVideos,playlistId
0,Nicholas Renotte,2019-01-26T22:31:46Z,295000,20574899,308,UUHXa4OpASJEwrHrLeIzw7Yg
1,Tech With Tim,2014-04-23T01:57:10Z,1670000,163402027,1312,UU4JX40jDee_tINbkjycV4Sg
2,Luke Barousse,2020-08-03T09:02:41.213077Z,498000,24836274,163,UULLw7jmFsvfIVaUFsLs8mlQ
3,Corey Schafer,2006-05-31T22:49:22Z,1400000,100423367,239,UUCezIgC97PvUuR4_gbFUs5g
4,Alex The Analyst,2020-01-08T05:04:24.970712Z,972000,45214678,344,UU7cs8q-gJRlGwj4A8OmCmXg
5,Ken Jee,2014-02-28T14:58:24Z,266000,9307997,288,UUiT9RITQ9PW6BhXK0y2jaeg
6,Data Professor,2019-08-17T15:59:56Z,200000,7156653,353,UUV8e2g4IWQqK71bbzGDEI4Q
7,Mo Chen,2022-12-25T20:25:38.187653Z,148000,5634083,215,UUDybamfye5An6p-j1t2YMsg
8,StatQuest with Josh Starmer,2011-05-24T01:52:48Z,1330000,74760977,285,UUtYLUTtgS3k1Fg4y5tAhLbw
9,Jay Feng,2019-11-19T19:16:30.516571Z,52000,3482872,420,UUcQx1UnmorvmSEZef4X7-6g
